In [2]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

# === 1. Cargar datos ===
ruta = './../Registros_sin_nulos.csv'  # Cambiar según sea necesario
df = pd.read_csv(ruta)

# === 2. Crear variable target ===
df['target'] = df['diasmora'].apply(lambda x: 1 if x > 7 else 0)

# Eliminar columnas que no se usarán
df = df.drop(columns=['nosocio', 'nocredito', 'sucursal', 'diasmora'], errors='ignore')

# === 3. Definir las 5 C’s ===
C5 = {
    'Carácter': ['reputación_localidad', 'exp_cred_externa', 'exp_cred_interna', 'renovadoestructurado'],
    'Capacidad': ['ingresos_ordinarios', 'num_dependientes', 'cap_pago', 'empleo'],
    'Capital': ['monto', 'tasaordinaria', 'tasamoratoria', 'deudas'],
    'Condiciones': ['producto', 'clasificacion', 'comercio_en_region', 'permiso_para_funcionar', 'plazocredito', 'frecuenciacapint'],
    'Colateral': ['montogarantialiq', 'montogarantiapre', 'montogarantiahipo']
}

# Filtrar solo variables existentes
C5 = {k: [v for v in vars_ if v in df.columns] for k, vars_ in C5.items()}

# === 4. Calcular importancia de variables (Mutual Information) ===
X = df.drop(columns=['target'])
y = df['target']

X_encoded = pd.get_dummies(X, drop_first=True)

mi = mutual_info_classif(X_encoded, y, discrete_features='auto', random_state=42)
pesos_df = pd.DataFrame({'Variable': X_encoded.columns, 'Peso': mi})
pesos_df = pesos_df.sort_values('Peso', ascending=False)

# Asociar cada variable a su categoría 5C
def categoria_5C(var):
    for c, vars_ in C5.items():
        if any(var.startswith(v + '_') or var == v for v in vars_):
            return c
    return 'Otras'

pesos_df['Categoria_5C'] = pesos_df['Variable'].apply(categoria_5C)

# === 5. Calcular riesgo por valor (promedio de morosos) ===
pesos_valores = []
for col in df.columns:
    if col not in ['target'] and df[col].dtype == 'object':
        tabla = df.groupby(col)['target'].mean().reset_index()
        tabla['Variable'] = col
        tabla.rename(columns={'target': 'Porcentaje_morosos', col: 'Valor'}, inplace=True)
        pesos_valores.append(tabla)

pesos_valores_df = pd.concat(pesos_valores, ignore_index=True) if pesos_valores else pd.DataFrame()

# === 6. Guardar resultados ===
pesos_df.to_csv('./pesos_variables.csv', index=False)
if not pesos_valores_df.empty:
    pesos_valores_df.to_csv('./pesos_valores.csv', index=False)

print('✅ Análisis completado.')
print(f'📁 Resultados guardados en: - pesos_variables.csv ({len(pesos_df)} filas)')
if not pesos_valores_df.empty:
    print(f'- pesos_valores.csv ({len(pesos_valores_df)} filas)')

✅ Análisis completado.
📁 Resultados guardados en: - pesos_variables.csv (184 filas)
- pesos_valores.csv (191 filas)
